In [1]:
import duckdb

cn = duckdb.connect(database=':memory:')

sql = """
    CREATE or replace view v_c2_detail as 
    select Number as stroke_number
        ,"Time (seconds)" as total_time
        ,"Distance (meters)" as total_distance
        ,"Pace (seconds)" as pace
        ,watts
        ,"Stroke Rate" as stroke_rate
        ,season as season_year
        ,"date" as workout_date
        ,machine_type
        ,strftime(
            TIMESTAMP '1970-01-01 00:00:00'
            + CAST(round("pace") AS INTEGER) * INTERVAL 1 SECOND,
            '%M:%S'
        ) AS total_time_mm_ss
        ,printf(
            '%d:%02d.%02d',
            CAST(floor(CAST(round("pace" * 100) AS INTEGER) / 6000) AS INTEGER),            -- minutes
            CAST(floor((CAST(round("pace" * 100) AS INTEGER) % 6000) / 100) AS INTEGER),    -- seconds
            CAST(CAST(round("pace" * 100) AS INTEGER) % 100 AS INTEGER)                     -- hundredths
        ) AS pace_mm_ss_ff
        ,CAST(regexp_extract(filename, '([0-9]+)\\.csv$', 1) AS BIGINT) AS log_id
    from read_csv_auto('~/concept2/workouts/*detail*.csv', filename=true)
"""


cn.execute(sql)

In [4]:
sql = """
    from v_c2_detail
    where workout_date >= '2026-01-19'
    and machine_type = 'rowerg'
    --limit 50
"""

cn.sql(sql)

┌───────────────┬────────────┬────────────────┬────────┬───────┬─────────────┬─────────────┬──────────────┬──────────────┬──────────────────┬───────────────┬───────────┐
│ stroke_number │ total_time │ total_distance │  pace  │ Watts │ stroke_rate │ season_year │ workout_date │ machine_type │ total_time_mm_ss │ pace_mm_ss_ff │  log_id   │
│     int64     │   double   │     double     │ double │ int64 │    int64    │    int64    │     date     │   varchar    │     varchar      │    varchar    │   int64   │
├───────────────┼────────────┼────────────────┼────────┼───────┼─────────────┼─────────────┼──────────────┼──────────────┼──────────────────┼───────────────┼───────────┤
│             1 │        0.0 │            0.0 │    0.0 │  NULL │        NULL │        2026 │ 2026-01-19   │ rowerg       │ 00:00            │ 0:00.00       │ 111482135 │
│             2 │        2.9 │           11.6 │  124.0 │   184 │          28 │        2026 │ 2026-01-19   │ rowerg       │ 02:04            │ 2:04.00 

In [12]:
sql = """
select *
from read_csv_auto('~/concept2/workouts/*summary*.csv')
where 1=1
    and date >= '2026-01-19'
    and Type = 'RowErg'
limit 5
"""
cn.sql(sql)

┌───────────┬─────────────────────┬──────────────────┬───────────────────────┬─────────────────────┬───────────────────────┬─────────────────────┬───────────────┬───────────────┬─────────────────────┬──────────────┬─────────┬───────────┬──────────┬───────────┬────────────────┬─────────────┬───────┬─────────┬─────────┬─────────┬──────────┬─────────────────────┬────────┐
│  Log ID   │        Date         │   Description    │ Work Time (Formatted) │ Work Time (Seconds) │ Rest Time (Formatted) │ Rest Time (Seconds) │ Work Distance │ Rest Distance │ Stroke Rate/Cadence │ Stroke Count │  Pace   │ Avg Watts │ Cal/Hour │ Total Cal │ Avg Heart Rate │ Drag Factor │  Age  │ Weight  │  Type   │ Ranked  │ Comments │    Date Entered     │ season │
│   int64   │      timestamp      │     varchar      │        varchar        │       double        │        varchar        │        int64        │     int64     │     int64     │        int64        │    int64     │ varchar │   int64   │  int64   │   int64

In [13]:
sql = """
SELECT
  DATE_TRUNC('year', timezone('UTC', "Date" :: timestamptz)) "c2_cube__workout_date_year",
  ROUND(sum("Work Distance") / 1000.0, 2) "c2_cube__total_distance_km"
FROM
  (
    SELECT
      *
    FROM
      read_csv_auto('~/concept2/workouts/*summary*.csv')
  ) AS "c2_cube"
GROUP BY
  1
ORDER BY
  1 ASC
LIMIT
  10000
"""
cn.sql(sql)

┌────────────────────────────┬────────────────────────────┐
│ c2_cube__workout_date_year │ c2_cube__total_distance_km │
│            date            │           double           │
├────────────────────────────┼────────────────────────────┤
│ 2024-01-01                 │                     594.99 │
│ 2025-01-01                 │                     961.27 │
│ 2026-01-01                 │                      73.63 │
└────────────────────────────┴────────────────────────────┘